In [ ]:

import sqlite3
import re
import math
from html.parser import HTMLParser
from collections import Counter, defaultdict

import requests
import time

# статьи Википедии
URLS = [
    "https://en.wikipedia.org/wiki/Cooking",
    "https://en.wikipedia.org/wiki/Baking",
    "https://en.wikipedia.org/wiki/Frying",
    "https://en.wikipedia.org/wiki/Boiling",
    "https://en.wikipedia.org/wiki/Simmering",
]


PAGES = {}

headers = {
    "User-Agent": "MiniSearchBot/1.0 (educational project; contact: your_email@example.com)"
}

for url in URLS:
    print(f"Загружаю: {url}")
    resp = requests.get(url, headers=headers, timeout=15)
    resp.raise_for_status()          # если ошибка HTTP — выбрасываем исключение
    PAGES[url] = resp.text

    time.sleep(1.0)

print(f"Загружено документов: {len(PAGES)}")


Загружаю: https://en.wikipedia.org/wiki/Cooking
Загружаю: https://en.wikipedia.org/wiki/Baking
Загружаю: https://en.wikipedia.org/wiki/Frying
Загружаю: https://en.wikipedia.org/wiki/Boiling
Загружаю: https://en.wikipedia.org/wiki/Simmering
Загружено документов: 5


In [ ]:

class LinkTextParser(HTMLParser):
    """
    Простой HTML-парсер:
    - собирает текст со страницы;
    - собирает значения href из ссылок <a>.
    Используется для реальных HTML-документов (статей Википедии).
    """
    def __init__(self):
        super().__init__()
        self.links = []
        self.text_chunks = []

    def handle_starttag(self, tag, attrs):
        if tag == "a":
            for name, value in attrs:
                if name == "href":
                    self.links.append(value)

    def handle_data(self, data):
        text = data.strip()
        if text:
            self.text_chunks.append(text)

def parse_html(html: str):
    """
    Возвращает:
    - text: объединённый текст страницы;
    - links: исходящие ссылки (сырой список href).
    """
    parser = LinkTextParser()
    parser.feed(html)
    text = " ".join(parser.text_chunks)
    links = parser.links
    return text, links

# Стоп-слова: немного русских и английских,
# чтобы отфильтровать общие «мусорные» слова.
STOPWORDS = {
    # Русские
    "и","в","во","на","как","по","о","об","от","за","из","к","ко",
    "а","но","для","что","это","этот","эта","эти","также","так",
    # Английские
    "the","and","or","for","of","in","on","at","is","are","to","a","an","by","with","as",
    "this","that","these","those","from","into","about","also","such","more","most",
}

def tokenize(text: str):
    """
    Токенизация текста:
    - приводим к нижнему регистру;
    - выбираем последовательности букв (латиница + кириллица);
    - отбрасываем короткие слова и стоп-слова.
    """
    tokens = re.findall(r"[A-Za-zА-Яа-яЁё]+", text.lower())
    return [t for t in tokens if t not in STOPWORDS and len(t) > 2]

# Быстрая проверка парсера на одной реальной странице
for url, html in PAGES.items():
    text, links = parse_html(html)
    print("URL:", url)
    print("  Текст (фрагмент):", text[:120].replace("\n", " "), "...")
    print("  Пример ссылок:", links[:5])
    break


URL: https://en.wikipedia.org/wiki/Cooking
  Текст (фрагмент): Cooking - Wikipedia (function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-langua ...
  Пример ссылок: ['#bodyContent', '/wiki/Main_Page', '/wiki/Wikipedia:Contents', '/wiki/Portal:Current_events', '/wiki/Special:Random']


In [ ]:

# Создание и заполнение базы данных SQLite

conn = sqlite3.connect(":memory:")
cur = conn.cursor()

cur.executescript("""
CREATE TABLE documents(
    id      INTEGER PRIMARY KEY AUTOINCREMENT,
    url     TEXT UNIQUE,
    title   TEXT,
    content TEXT
);

CREATE TABLE terms(
    id      INTEGER PRIMARY KEY AUTOINCREMENT,
    term    TEXT UNIQUE
);

CREATE TABLE doc_terms(
    term_id INTEGER,
    doc_id  INTEGER,
    tf      INTEGER,
    PRIMARY KEY(term_id, doc_id)
);

CREATE TABLE links(
    from_doc INTEGER,
    to_doc   INTEGER
);
""")

for url, html in PAGES.items():
    text, links = parse_html(html)
    # Заголовок вытащим как первое предложение (фрагмент текста)
    title = text.split(".")[0][:200]
    cur.execute(
        "INSERT INTO documents(url, title, content) VALUES (?, ?, ?)",
        (url, title, text)
    )

conn.commit()

# Карта url -> id
cur.execute("SELECT id, url FROM documents")
url_to_id = {url: doc_id for doc_id, url in cur.fetchall()}

# Подготовим вспомогательную функцию нормализации wiki-ссылок

def normalize_wiki_link(href: str):
    if not href:
        return None
    # обрежем якоря и параметры
    href = href.split("#")[0].split("?")[0]
    href = href.strip()
    if not href:
        return None

    # относительная ссылка вида /wiki/Baking
    if href.startswith("/wiki/"):
        return "https://en.wikipedia.org" + href

    # абсолютная ссылка вида https://en.wikipedia.org/wiki/Baking
    if href.startswith("https://en.wikipedia.org/wiki/") or href.startswith("http://en.wikipedia.org/wiki/"):
        return href

    # протокол-независимая ссылка //en.wikipedia.org/wiki/...
    if href.startswith("//en.wikipedia.org/wiki/"):
        return "https:" + href

    # всё остальное (внешние сайты и т.п.) нас не интересует
    return None

# Парсинг, токенизация и заполнение terms / doc_terms / links для реальных страниц
for url, html in PAGES.items():
    doc_id = url_to_id[url]
    text, links = parse_html(html)
    tokens = tokenize(text)
    counts = Counter(tokens)

    # Термины (слова)
    for term, tf in counts.items():
        cur.execute("INSERT OR IGNORE INTO terms(term) VALUES (?)", (term,))
        cur.execute("SELECT id FROM terms WHERE term = ?", (term,))
        term_id = cur.fetchone()[0]

        cur.execute("""
            INSERT OR REPLACE INTO doc_terms(term_id, doc_id, tf)
            VALUES (?, ?, ?)
        """, (term_id, doc_id, tf))

    # Ссылки между пятью статьями
    for href in links:
        norm = normalize_wiki_link(href)
        # Оставляем только ссылки, которые попадают в наш корпус
        if norm and norm in url_to_id:
            to_id = url_to_id[norm]
            cur.execute(
                "INSERT INTO links(from_doc, to_doc) VALUES (?, ?)",
                (doc_id, to_id)
            )

conn.commit()

# Быстрая проверка заполнения
cur.execute("SELECT COUNT(*) FROM documents")
print("Документов:", cur.fetchone()[0])
cur.execute("SELECT COUNT(*) FROM terms")
print("Терминов:", cur.fetchone()[0])
cur.execute("SELECT COUNT(*) FROM doc_terms")
print("Записей doc_terms:", cur.fetchone()[0])
cur.execute("SELECT from_doc, to_doc FROM links")
print("Ссылки (from_doc -> to_doc):", cur.fetchall())


Документов: 5
Терминов: 4901
Записей doc_terms: 8811
Ссылки (from_doc -> to_doc): [(1, 1), (1, 1), (1, 1), (1, 2), (1, 4), (1, 4), (1, 5), (1, 4), (1, 2), (1, 5), (1, 3), (1, 2), (1, 4), (1, 5), (1, 3), (2, 2), (2, 2), (2, 2), (2, 1), (2, 4), (2, 5), (2, 3), (3, 3), (3, 3), (3, 3), (3, 1), (3, 1), (3, 2), (3, 4), (3, 5), (4, 4), (4, 4), (4, 4), (4, 1), (4, 1), (4, 5), (4, 1), (4, 2), (4, 5), (4, 3), (5, 5), (5, 5), (5, 5), (5, 1), (5, 4), (5, 1), (5, 2), (5, 4), (5, 3)]


In [ ]:

# Построение графа ссылок для PageRank

cur.execute("SELECT id FROM documents")
doc_ids = [row[0] for row in cur.fetchall()]
N = len(doc_ids)

# Список смежности: doc_id -> список doc_id, на которые он ссылается
adj = {doc_id: [] for doc_id in doc_ids}

cur.execute("SELECT from_doc, to_doc FROM links")
for from_doc, to_doc in cur.fetchall():
    adj[from_doc].append(to_doc)

print("Список смежности (adjacency) по реальным статьям:")
for doc_id in sorted(adj):
    print(f"  {doc_id} -> {adj[doc_id]}")


Список смежности (adjacency) по реальным статьям:
  1 -> [1, 1, 1, 2, 4, 4, 5, 4, 2, 5, 3, 2, 4, 5, 3]
  2 -> [2, 2, 2, 1, 4, 5, 3]
  3 -> [3, 3, 3, 1, 1, 2, 4, 5]
  4 -> [4, 4, 4, 1, 1, 5, 1, 2, 5, 3]
  5 -> [5, 5, 5, 1, 4, 1, 2, 4, 3]


In [ ]:

# PageRank через модель MapReduce

def pagerank_mapreduce(adj, num_iters=20, damping=0.85):
    """
    adj: словарь doc_id -> список doc_id, на которые ведут исходящие ссылки.

    Логика MapReduce:
    - "Map": каждая страница делит свой текущий PageRank между всеми ссылками;
    - "Reduce": для каждой страницы суммируем все вклады и применяем формулу PageRank.
    """
    nodes = list(adj.keys())
    N = len(nodes)
    ranks = {node: 1.0 / N for node in nodes}  # начальное равномерное распределение

    for it in range(num_iters):
        contributions = []

        # Map: генерируем вклады от каждой вершины к соседям
        for node in nodes:
            out_links = adj[node]
            if out_links:
                share = ranks[node] / len(out_links)
                for dest in out_links:
                    contributions.append((dest, share))
            else:
                # "висячая" страница: равномерно раздаём её ранг всем
                share = ranks[node] / N
                for dest in nodes:
                    contributions.append((dest, share))

        # Reduce: суммируем вклады по каждой вершине
        new_ranks = {node: 0.0 for node in nodes}
        for dest, value in contributions:
            new_ranks[dest] += value

        # Применяем damping-фактор (коэффициент затухания)
        for node in nodes:
            new_ranks[node] = (1 - damping) / N + damping * new_ranks[node]

        ranks = new_ranks

    return ranks

def print_pagerank(pr_dict, label):
    print(label)
    for doc_id, pr in sorted(pr_dict.items(), key=lambda x: x[1], reverse=True):
        cur.execute("SELECT url, title FROM documents WHERE id = ?", (doc_id,))
        url, title = cur.fetchone()
        print(f"  doc_id={doc_id}, PR={pr:.4f}")
        print(f"     url={url}")
        print(f"     title={title[:80]}...")
    print()

pr_mr = pagerank_mapreduce(adj, num_iters=20)
print_pagerank(pr_mr, "PageRank (MapReduce) для статей:")


PageRank (MapReduce) для статей:
  doc_id=1, PR=0.2199
     url=https://en.wikipedia.org/wiki/Cooking
     title=Cooking - Wikipedia (function(){var className="client-js vector-feature-language...
  doc_id=4, PR=0.2143
     url=https://en.wikipedia.org/wiki/Boiling
     title=Boiling - Wikipedia (function(){var className="client-js vector-feature-language...
  doc_id=5, PR=0.2028
     url=https://en.wikipedia.org/wiki/Simmering
     title=Simmering - Wikipedia (function(){var className="client-js vector-feature-langua...
  doc_id=2, PR=0.1932
     url=https://en.wikipedia.org/wiki/Baking
     title=Baking - Wikipedia (function(){var className="client-js vector-feature-language-...
  doc_id=3, PR=0.1699
     url=https://en.wikipedia.org/wiki/Frying
     title=Frying - Wikipedia (function(){var className="client-js vector-feature-language-...



In [ ]:

# PageRank в стиле Pregel


def pagerank_pregel(adj, num_iters=20, damping=0.85):
    """
    Pregel-подход:
    - каждая вершина на супер-шаге рассылает свой ранг соседям;
    - вершина суммирует все полученные сообщения и пересчитывает PageRank.
    """
    nodes = list(adj.keys())
    N = len(nodes)
    ranks = {node: 1.0 / N for node in nodes}

    for it in range(num_iters):
        # Словарь "сообщений" для каждой вершины
        messages = {node: 0.0 for node in nodes}

        # Вершины рассылают свой текущий PageRank
        for node in nodes:
            out_links = adj[node]
            if out_links:
                share = ranks[node] / len(out_links)
                for dest in out_links:
                    messages[dest] += share
            else:
                # висячая вершина: делим ранг между всеми
                share = ranks[node] / N
                for dest in nodes:
                    messages[dest] += share

        # Каждая вершина пересчитывает свой PageRank
        for node in nodes:
            ranks[node] = (1 - damping) / N + damping * messages[node]

    return ranks

pr_pregel = pagerank_pregel(adj, num_iters=20)
print_pagerank(pr_pregel, "PageRank (Pregel) для статей:")

print("Сравнение MapReduce и Pregel по doc_id:")
for doc_id in sorted(adj.keys()):
    print(f"  doc_id={doc_id}: MR={pr_mr[doc_id]:.4f}, Pregel={pr_pregel[doc_id]:.4f}")


PageRank (Pregel) для статей:
  doc_id=1, PR=0.2199
     url=https://en.wikipedia.org/wiki/Cooking
     title=Cooking - Wikipedia (function(){var className="client-js vector-feature-language...
  doc_id=4, PR=0.2143
     url=https://en.wikipedia.org/wiki/Boiling
     title=Boiling - Wikipedia (function(){var className="client-js vector-feature-language...
  doc_id=5, PR=0.2028
     url=https://en.wikipedia.org/wiki/Simmering
     title=Simmering - Wikipedia (function(){var className="client-js vector-feature-langua...
  doc_id=2, PR=0.1932
     url=https://en.wikipedia.org/wiki/Baking
     title=Baking - Wikipedia (function(){var className="client-js vector-feature-language-...
  doc_id=3, PR=0.1699
     url=https://en.wikipedia.org/wiki/Frying
     title=Frying - Wikipedia (function(){var className="client-js vector-feature-language-...

Сравнение MapReduce и Pregel по doc_id:
  doc_id=1: MR=0.2199, Pregel=0.2199
  doc_id=2: MR=0.1932, Pregel=0.1932
  doc_id=3: MR=0.1699, Pregel=0.169

In [ ]:

# Построение инвертированного индекса


# term_id <-> term
cur.execute("SELECT id, term FROM terms")
term_id_to_term = {tid: term for tid, term in cur.fetchall()}
term_to_id = {term: tid for tid, term in term_id_to_term.items()}

# inverted_index: term -> список (doc_id, tf)
cur.execute("SELECT term_id, doc_id, tf FROM doc_terms")
inverted_index = {}
doc_lengths = {doc_id: 0 for doc_id in doc_ids}

for term_id, doc_id, tf in cur.fetchall():
    term = term_id_to_term[term_id]
    inverted_index.setdefault(term, []).append((doc_id, tf))
    doc_lengths[doc_id] += tf

# сортируем списки по doc_id
for term in inverted_index:
    inverted_index[term].sort(key=lambda x: x[0])

# DF и IDF для TF-IDF-скоринга
N = len(doc_ids)
df = {term: len(postings) for term, postings in inverted_index.items()}
idf = {term: math.log(N / df_t) if df_t > 0 else 0.0
       for term, df_t in df.items()}

print("Пример записей инвертированного индекса (реальные статьи):")
for term, postings in list(inverted_index.items())[:10]:
    print(f"  term='{term}': postings={postings}")


Пример записей инвертированного индекса (реальные статьи):
  term='cooking': postings=[(1, 134), (2, 23), (3, 24), (4, 21), (5, 21)]
  term='wikipedia': postings=[(1, 19), (2, 15), (3, 16), (4, 16), (5, 14)]
  term='function': postings=[(1, 6), (2, 7), (3, 6), (4, 7), (5, 6)]
  term='var': postings=[(1, 9), (2, 7), (3, 9), (4, 7), (5, 7)]
  term='classname': postings=[(1, 5), (2, 5), (3, 5), (4, 5), (5, 5)]
  term='client': postings=[(1, 3), (2, 3), (3, 3), (4, 3), (5, 3)]
  term='vector': postings=[(1, 17), (2, 17), (3, 17), (4, 17), (5, 17)]
  term='feature': postings=[(1, 11), (2, 10), (3, 10), (4, 10), (5, 10)]
  term='language': postings=[(1, 5), (2, 6), (3, 2), (4, 2), (5, 4)]
  term='header': postings=[(1, 7), (2, 3), (3, 3), (4, 3), (5, 3)]


In [ ]:

# Полнотекстовый поиск: term-at-a-time и document-at-a-time


def normalize_query_terms(query: str):
    """
    Нормализация текста запроса:
    используем ту же токенизацию и фильтрацию, что и для документов.
    """
    return tokenize(query)

def search_taat(query: str, top_k=5):
    """
    Term-at-a-time:
    - обрабатываем термины запроса по одному;
    - по каждому термину проходим его postings-список и обновляем скор документов.
    """
    terms_q = normalize_query_terms(query)
    scores = defaultdict(float)

    for term in terms_q:
        postings = inverted_index.get(term, [])
        w_t = idf.get(term, 0.0)
        for doc_id, tf in postings:
            scores[doc_id] += tf * w_t  # TF-IDF вклад

    # Смешиваем текстовый скор с PageRank
    if scores:
        max_pr = max(pr_mr.values())
        for doc_id in scores:
            pr_norm = pr_mr[doc_id] / max_pr
            scores[doc_id] = 0.7 * scores[doc_id] + 0.3 * pr_norm

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

def search_daat(query: str, top_k=5):
    """
    Document-at-a-time:
    - одновременно идём по postings-спискам всех термов;
    - обрабатываем документы по одному, собирая вклад всех термов.
    """
    terms_q = normalize_query_terms(query)
    postings_lists = [inverted_index.get(t, []) for t in terms_q]
    pointers = [0] * len(postings_lists)
    scores = {}

    if not postings_lists:
        return []

    max_pr = max(pr_mr.values())

    while True:
        current_ids = []
        for i, lst in enumerate(postings_lists):
            if pointers[i] < len(lst):
                current_ids.append(lst[pointers[i]][0])

        if not current_ids:
            break

        doc_id = min(current_ids)
        score = 0.0

        # для текущего doc_id собираем вклад всех термов запроса
        for i, lst in enumerate(postings_lists):
            if pointers[i] < len(lst) and lst[pointers[i]][0] == doc_id:
                tf = lst[pointers[i]][1]
                term = terms_q[i]
                score += tf * idf.get(term, 0.0)
                pointers[i] += 1

        pr_norm = pr_mr[doc_id] / max_pr
        score = 0.7 * score + 0.3 * pr_norm
        scores[doc_id] = score

    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

def pretty_print_results(results, label):
    print(label)
    if not results:
        print("  Нет результатов.")
        return
    for rank, (doc_id, score) in enumerate(results, start=1):
        cur.execute("SELECT url, title FROM documents WHERE id = ?", (doc_id,))
        url, title = cur.fetchone()
        print(f"  {rank}. doc_id={doc_id}, score={score:.4f}, PR={pr_mr[doc_id]:.3f}")
        print(f"     url={url}")
        print(f"     title={title[:100]}...")
    print()


In [ ]:

# Демонстрация конвейера поиска


# Пример запроса: слова по тематике кулинарии и техник приготовления
query = "cooking baking frying"

results_taat = search_taat(query, top_k=5)
results_daat = search_daat(query, top_k=5)

pretty_print_results(results_taat, f"Term-at-a-time (запрос: '{query}')")
pretty_print_results(results_daat, f"Document-at-a-time (запрос: '{query}')")


Term-at-a-time (запрос: 'cooking baking frying')
  1. doc_id=1, score=0.3000, PR=0.220
     url=https://en.wikipedia.org/wiki/Cooking
     title=Cooking - Wikipedia (function(){var className="client-js vector-feature-language-in-header-enabled v...
  2. doc_id=4, score=0.2924, PR=0.214
     url=https://en.wikipedia.org/wiki/Boiling
     title=Boiling - Wikipedia (function(){var className="client-js vector-feature-language-in-header-enabled v...
  3. doc_id=5, score=0.2766, PR=0.203
     url=https://en.wikipedia.org/wiki/Simmering
     title=Simmering - Wikipedia (function(){var className="client-js vector-feature-language-in-header-enabled...
  4. doc_id=2, score=0.2635, PR=0.193
     url=https://en.wikipedia.org/wiki/Baking
     title=Baking - Wikipedia (function(){var className="client-js vector-feature-language-in-header-enabled ve...
  5. doc_id=3, score=0.2318, PR=0.170
     url=https://en.wikipedia.org/wiki/Frying
     title=Frying - Wikipedia (function(){var className="client-js